In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-stock-prices-may-2017-to-may-2022/AMZN.csv


### Loading Data and Set-Up

In [5]:
data = pd.read_csv('/kaggle/input/amazon-stock-prices-may-2017-to-may-2022/AMZN.csv')

In [6]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-06-01,49.929501,49.949501,49.568501,49.797501,49.797501,49096000
1,2017-06-02,49.949501,50.424000,49.783501,50.336498,50.336498,75046000
2,2017-06-05,50.361500,50.660500,50.175499,50.567001,50.567001,54398000
3,2017-06-06,50.599998,50.825001,50.062500,50.150002,50.150002,66928000
4,2017-06-07,50.297501,50.512501,50.099998,50.503502,50.503502,56460000


In [7]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1.259000e+03
mean,112.712150,113.962250,111.290146,112.638289,112.638289,8.661014e+07
std,41.589328,42.096757,41.022137,41.521249,41.521249,4.156922e+07
min,47.000000,47.431499,46.349998,46.930000,46.930000,1.762600e+07
25%,82.564251,83.682998,81.300998,82.778748,82.778748,5.843600e+07
50%,95.899002,96.611504,94.828499,95.449501,95.449501,7.475200e+07
75%,158.343750,159.924003,156.199997,158.086998,158.086998,1.025520e+08
max,187.199997,188.654007,184.839493,186.570496,186.570496,3.313000e+08


## Feature Engineering
- Adding date and time based features
- Adding rolling mean averages for close, high, low, volume
- Adding lag features for close 
- Removing columns from data set to avoid leakage - the purpose of the model is to predict the given day's stock close, and so data from that day (High, Low, etc.) would not be available yet for predictions. 

In [8]:

# adding date based features: day of week, day of month, day of year 

data['Date'] = pd.to_datetime(data['Date'])
data['Day of week'] = data['Date'].dt.weekday
data['Day of month'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year

# rolling averages for days = 7, 30, 90, 180, 365

data["7d Mean Close"] = data['Close'].rolling(window = 5).mean() # 5 business days = 1 wk
data["30d Mean Close"] = data['Close'].rolling(window = 20).mean() # 20 business days = 1 mo
data["90d Mean Close"] = data['Close'].rolling(window = 60).mean() # 60 business days = 3 mo
data["120d Mean Close"] = data['Close'].rolling(window = 120).mean() #120 business days = 6 mo

# adding lag features: Close on (date - 1), (date-2), 3,4,5,7,14,21,30,60,90,180, 365
data['Close-1'] = data['Close'].shift(periods=1)
data['Close-2'] = data['Close'].shift(periods=2)
data['Close-3'] = data['Close'].shift(periods=3)
data['Close-4'] = data['Close'].shift(periods=4)
data['Close-5'] = data['Close'].shift(periods=5)
data['Close-7'] = data['Close'].shift(periods=7)
data['Close-14'] = data['Close'].shift(periods=14)
data['Close-21'] = data['Close'].shift(periods=21)
data['Close-30'] = data['Close'].shift(periods=30)
data['Close-60'] = data['Close'].shift(periods=60)
data['Close-90'] = data['Close'].shift(periods=90)

# Adding rolling averages for high, low and volume
data["7d Mean High"] = data['High'].rolling(window = 5).mean() # 5 business days = 1 wk
data["30d Mean High"] = data['High'].rolling(window = 20).mean() # 20 business days = 1 mo
data["90d Mean High"] = data['High'].rolling(window = 60).mean() # 60 business days = 3 mo
data["120d Mean High"] = data['High'].rolling(window = 120).mean() #120 business days = 6 mo

data["7d Mean Low"] = data['Low'].rolling(window = 5).mean() # 5 business days = 1 wk
data["30d Mean Low"] = data['Low'].rolling(window = 20).mean() # 20 business days = 1 mo
data["90d Mean Low"] = data['Low'].rolling(window = 60).mean() # 60 business days = 3 mo
data["120d Mean Low"] = data['Low'].rolling(window = 120).mean() #120 business days = 6 mo

data["7d Mean Volume"] = data['Volume'].rolling(window = 5).mean() # 5 business days = 1 wk
data["30d Mean Volume"] = data['Volume'].rolling(window = 20).mean() # 20 business days = 1 mo
data["90d Mean Volume"] = data['Volume'].rolling(window = 60).mean() # 60 business days = 3 mo
data["120d Mean Volume"] = data['Volume'].rolling(window = 120).mean() #120 business days = 6 mo


# Will only train models on data with all features intact, so will exclude the first 120 data points
full_data = data.drop(index = range(119)) 
full_data = full_data.reset_index(drop=True)

#### Dropping Features to Limit Data Leakage

In [15]:
# To avoid leakage must drop features that will be unavailable at time of prediction
X = full_data.drop(columns = ['Close', 'Adj Close', 'Open', 'High', 'Low', 'Volume']) 
Y = full_data['Close']
# Adding index in place of date
X ['index'] = X.index
# Removing dates so XGBoost will work
X_no_dates = X.drop(columns = ['Date']).to_numpy()
Y_no_dates = Y.drop(columns = ['Date']).to_numpy()


#### Creating a simple XGBoost Regression Model 
This will have a high error value but will serve as a comparison for the cross-validated regression and prophet models

In [19]:
## Creating a manual split (must allocate indexes manually to prevent shuffling)
X_train = X_no_dates[0:800]
X_test = X_no_dates[800:1139]
Y_train = Y_no_dates[0:800]
Y_test = Y_no_dates[800:1139]


In [20]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
model = XGBRegressor()
model.fit(X_train, Y_train)
preds = model.predict(X_test)

mse_xgboost = mean_squared_error (preds, Y_test)
print(mse_xgboost)


76.80376326906087


# Adding Cross Validation using a Rolling 120-day Window 
Using an rolling window to train the model (implementing manually as having issues importing sktime to env), generates folds across an rolling window. The length of the training series remains constant, with each subsequent fold retaining the 120 bus days history up to that point. The testing series remains the same length throughout. 

In [25]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from statistics import mean

i = 0
all_mse = []

for train_index, test_index in tscv.split(X):
    while i+180 < len(X):
        X_train, X_test = X_no_dates[i:i+120], X_no_dates[i+120:i+180]
        Y_train, Y_test = Y_no_dates[i:i+120], Y_no_dates[i+120:i+180]
        model = XGBRegressor()
        model.fit(X_train, Y_train)
        preds = model.predict(X_test)
        mse = mean_squared_error (Y_test, preds)
        all_mse.append(mse)
        i = i + 60

print(mean(all_mse))
    
    
#for train_index, test_index in tscv.split(X):
#    cv_train, cv_test = 
    





67.72227610136395


# Facebook Prophet Approach 

In [ ]:
!pip install prophet;

In [ ]:
from prophet import Prophet
# dropping cols to avoid leakage
full_data_prophet = full_data.drop(columns = ['Adj Close']) 
full_data_prophet.rename(columns={'Date':'ds','Close':'y'},inplace=True)
full_data_prophet['ds'] = pd.to_datetime(full_data_prophet['ds'])


In [ ]:
# we will use a rolling 120 day prophet model to predict the following day 
from sklearn.metrics import mean_squared_error
preds_vs_actual = pd.DataFrame(columns = ['Actual', 'Predicted'], index = [0,1,2,3,4,5,6,7,8,9])

for i in range(0,10): #for i in range(0,len(full_data_prophet)):
    rolling_train_data = full_data_prophet.iloc[0+i:240+i,:] #0-120, 1-121, ...
    model = Prophet()
    model.fit(full_data_prophet)
    days_left = len(full_data_prophet) - i 
    future_dates = model.make_future_dataframe(periods=1, freq='D') # MUST CHANGE to get only next day pred
    forecast = model.predict(future_dates)
    preds_vs_actual['Actual'][i] = Y[i]
    preds_vs_actual['Predicted'][i]= forecast['yhat'][0]
    i=i+1
    
preds_vs_actual.head()
mse = mean_squared_error (preds_vs_actual['Actual'], preds_vs_actual['Predicted'])

In [ ]:
print(mse)